In [1]:
%connect "AOA DEMO"

Password: ··········


Success: 'AOA DEMO' connection established and activated for user 'AOA_DEMO'


In [5]:
select top 1 model_id, model_version, deployed_at from aoa_ivsm_models order by deployed_at desc

model_id,model_version,deployed_at
2baa77c4-e63f-4213-a59d-89e8545024e4,81efb953-7c72-494d-a404-363ad4aa4352,2021-08-12 13:36:16.050000


In [1]:
-- issue with the BYOM predict in that it produces some incorrect predictions for this model.. Might be version of ivsm.
-- for now just filter them out 
SELECT cust_id, y_test, CAST(y_pred AS INT) FROM (
    SELECT cust_id, cc_acct_ind as y_test, CAST(score_result AS JSON).JSONExtractValue('$.predicted_cc_acct_ind') as y_pred FROM IVSM.IVSM_SCORE(
                ON (SELECT * FROM bank_features_test_v) AS DataTable
                ON (SELECT model_id, model FROM aoa_ivsm_models WHERE model_version = '81efb953-7c72-494d-a404-363ad4aa4352') AS ModelTable DIMENSION
                USING
                    ModelID('2baa77c4-e63f-4213-a59d-89e8545024e4')
                    ColumnsToPreserve('cust_id', 'cc_acct_ind')
                    ModelType('PMML')
            ) sc
    ) T WHERE T.y_pred=0 OR T.y_pred=1;

ERROR: No active connection